In [1]:
from __future__ import print_function, division

# BERT
import torch
from torch.nn import CosineSimilarity
import numpy as np

# XLNet
import transformers

# Paper
import json
import random

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Similarity function

In [2]:
def similarity(words, encodings, word_idx, measure="cos"):
    cos = CosineSimilarity(dim=0)
    similarities = []
    print("Similarity with \'"+words[word_idx]+"\'")
    for i in range(len(words)):
        # cos or torch.dist
        if measure == "cos":
            sim = cos(encodings[i], encodings[word_idx])
            print(words[i]+"\t"+str(sim.item()))
        elif measure == "euclidean":
            sim = torch.dist(encodings[i], encodings[word_idx])
            print(words[i]+"\t"+str(sim.item()))
        else:
            print("Unknown distance metric")

## BERT

In [11]:
# bert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
# bert_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
bert_model = transformers.BertModel.from_pretrained('bert-base-cased')

def w2BERT(words, tokenizer=bert_tokenizer, model=bert_model):
    # Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
    indexed_tokens = tokenizer.encode(" ".join(words), add_special_tokens=False)
    for i, idx in enumerate(indexed_tokens):
        if idx == 100:
            print(words[i]+" is unknown")

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor)
        print(encoded_layers.shape)
    return encoded_layers[0]

def w2BERT2(words, tokenizer=bert_tokenizer, model=bert_model):
    encodings = torch.Tensor(len(words), 768)
    for i, word in enumerate(words):
        indexed_tokens = tokenizer.encode(word, add_special_tokens=True)
        if 100 in indexed_tokens: print(word+" is unknown")
        print(tokenizer.convert_ids_to_tokens(indexed_tokens))
        with torch.no_grad():
            tokens_tensor = torch.tensor([indexed_tokens])
            encoded_layers = model(tokens_tensor)[0]
        encodings[i,:] = encoded_layers[0,0]
    encodings = encodings / encodings.norm(dim=1)[:, None]
    return encodings

print(bert_tokenizer.all_special_ids)
print(bert_tokenizer.all_special_tokens)

[101, 0, 103, 100, 102]
['[CLS]', '[PAD]', '[MASK]', '[UNK]', '[SEP]']


In [12]:
words = ["man", "woman", "car", "tractor", "bee", "vehicle", "motor", "potato", "carrot", "food", "metal", "she", "he", "programmer", "computer"]
encodings = w2BERT2(words)
print(encodings.shape)

similarity(words, encodings,6)

['[CLS]', 'man', '[SEP]']
(tensor([[[ 0.4628, -0.1432,  0.0398,  ..., -0.1443,  0.3387, -0.0368],
         [ 0.7357, -0.8538,  0.2751,  ..., -0.4005,  0.3785, -0.0588],
         [ 1.4516, -0.4821, -0.8175,  ..., -0.1294,  1.0990, -0.2170]]]), tensor([[-0.7187,  0.4139,  0.9999, -0.9945,  0.9638,  0.8383,  0.9832, -0.9723,
         -0.9790, -0.5187,  0.9843,  0.9991, -0.9909, -0.9998,  0.6829, -0.9781,
          0.9861, -0.5522, -1.0000, -0.6345, -0.3825, -0.9999,  0.3836,  0.9595,
          0.9815,  0.0098,  0.9877,  1.0000,  0.8148,  0.1581,  0.3823, -0.9916,
          0.7170, -0.9989,  0.1441,  0.1302,  0.2561, -0.2670,  0.4982, -0.9063,
         -0.6838, -0.1601,  0.7203, -0.4586,  0.7653,  0.1111,  0.1433, -0.0395,
         -0.2101,  0.9999, -0.9645,  0.9993, -0.9795,  0.9974,  0.9968,  0.2899,
          0.9967,  0.1308, -0.9942,  0.4293,  0.9695,  0.1057,  0.9032, -0.3748,
          0.0913, -0.4728, -0.7935,  0.0472, -0.3879, -0.0241,  0.2885,  0.4127,
          0.9881, -0.8863, -

TypeError: tuple indices must be integers or slices, not tuple

## XLNet

In [5]:
xlnet_tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = transformers.XLNetModel.from_pretrained('xlnet-base-cased')

def w2xlnet(words, tokenizer=xlnet_tokenizer, model=xlnet_model):
    indexed_tokens = tokenizer.encode("<sep>".join(words), add_special_tokens=True)[:-1] # Exclude final <cls>
    for i, idx in enumerate(indexed_tokens):
        if idx == 0:
            print(words[i]+" is unknown")
    print(tokenizer.convert_ids_to_tokens(indexed_tokens))

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
    return encoded_layers[0]

def w2xlnet2(words, tokenizer=xlnet_tokenizer, model=xlnet_model):
    encodings = torch.Tensor(len(words), 768)
    for i, word in enumerate(words):
        indexed_tokens = tokenizer.encode(word, add_special_tokens=True)
        if 0 in indexed_tokens: print(word+" is unknown")
        print(tokenizer.convert_ids_to_tokens(indexed_tokens))
        with torch.no_grad():
            tokens_tensor = torch.tensor([indexed_tokens])
            encoded_layers = model(tokens_tensor)[0]
        encodings[i,:] = encoded_layers[0,0]
    encodings = encodings / encodings.norm(dim=1)[:, None]
    return encodings

print(xlnet_tokenizer.all_special_ids)
print(xlnet_tokenizer.all_special_tokens)

[3, 2, 7, 5, 6, 0, 4, 8, 1]
['<cls>', '</s>', '<eod>', '<pad>', '<mask>', '<unk>', '<sep>', '<eop>', '<s>']


In [6]:
words = ["man", "woman", "farmer", "tractor", "flower", "car", "vehicle", "motor", "bee", "potato", "carrot", "food", "metal", "she", "he", "programmer", "computer"]
encodings = w2xlnet2(words)
print(encodings.shape)

similarity(words, encodings,3, measure="cos")

['▁man', '<sep>', '<cls>']
['▁woman', '<sep>', '<cls>']
['▁farmer', '<sep>', '<cls>']
['▁tractor', '<sep>', '<cls>']
['▁flower', '<sep>', '<cls>']
['▁car', '<sep>', '<cls>']
['▁vehicle', '<sep>', '<cls>']
['▁motor', '<sep>', '<cls>']
['▁be', 'e', '<sep>', '<cls>']
['▁potato', '<sep>', '<cls>']
['▁carrot', '<sep>', '<cls>']
['▁food', '<sep>', '<cls>']
['▁metal', '<sep>', '<cls>']
['▁she', '<sep>', '<cls>']
['▁he', '<sep>', '<cls>']
['▁programmer', '<sep>', '<cls>']
['▁computer', '<sep>', '<cls>']
torch.Size([17, 768])
Similarity with 'tractor'
man	0.9632200002670288
woman	0.9631847739219666
farmer	0.9791725277900696
tractor	1.0
flower	0.9555322527885437
car	0.9815327525138855
vehicle	0.9430563449859619
motor	0.9505726099014282
bee	0.9657217264175415
potato	0.9759369492530823
carrot	0.9839041233062744
food	0.9652472734451294
metal	0.9675664305686951
she	0.9368990063667297
he	0.9590157866477966
programmer	0.9410752058029175
computer	0.9757446646690369


## XLNet words to embeddings

In [7]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423


In [8]:
def save_w2XLNet(words, filename):
    xlnet_tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
    xlnet_model = transformers.XLNetModel.from_pretrained('xlnet-base-cased')
    unknowns = []
    with open(filename, "w") as f:
        for i, word in enumerate(words):
            if i % 100 == 0:
                print(i)
            f.write(word+" ")
            indexed_tokens = xlnet_tokenizer.encode(word, add_special_tokens=False)
            if 0 in indexed_tokens:
                print(word+" is unknown")
                unknowns.append(word)
    #       print(tokenizer.convert_ids_to_tokens(indexed_tokens))
            with torch.no_grad():
                tokens_tensor = torch.tensor([indexed_tokens])
                encoded_layers = xlnet_model(tokens_tensor)[0][0,0] # Tensor
                vector_list = encoded_layers.tolist()
                vector_list = [str(x) for x in vector_list]
                f.write(" ".join(vector_list)+"\n")

In [11]:
save_w2XLNet(words, "xlnet_specials.csv")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18